# Blue Brain BioExplorer - Neuromodulation movie
![](../bioexplorer_neuromodulation_banner.png)

In [ ]:
from bioexplorer import BioExplorer, Protein, Volume, Vector3, Quaternion, MolecularSystemAnimationParams
import os

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
be.reset_scene()
be.set_general_settings(model_visibility_on_creation=False)

In [ ]:
data_folder = os.getenv('NEUROMODULATION_DATA_FOLDER')

In [ ]:
load_neuron = True
load_mesh_neuron = True
neuron_guid = 989
load_astrocytes = True
load_vasculature = True
load_microtubules = True
use_sdf = True
generate_movie = False

scale = Vector3(1000.0, 1000.0, 1000.0)
scene_center = [552.193724,1736.099481,235.409219]

In [ ]:
neuron_population_name = 'o1'
vasculature_population_name = 'vasculature'
astrocytes_population_name = 'astrocytes'

if load_neuron:
    neuron_sql_filter = 'guid=%d' % neuron_guid
    neuron_assembly_name = 'Neuron'
    be.remove_assembly(neuron_assembly_name)
    be.add_assembly(neuron_assembly_name)
    be.add_neurons(
        assembly_name=neuron_assembly_name,
        morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
        population_name=neuron_population_name,
        use_sdf=use_sdf, generate_buttons=True,
        load_synapses=True, show_membrane=not load_mesh_neuron,
        generate_internals=True, generate_externals=False,
        sql_node_filter=neuron_sql_filter, scale=scale,
        animation_params=MolecularSystemAnimationParams(1))

if load_mesh_neuron:
    neuron_mesh_model = core.add_model(
        name='Neuron',
        path=os.path.join(data_folder, 'meshes', 'vd100621_idB_-_Scale_x1.000_y1.025_z1.000_-_Clone_6.obj'))
    tf = {
        'rotation': [0, -0.87495604080936, 0, 0.48420236126149724],
        'rotation_center': [0.0, 0.0, 0.0],
        'scale': [scale.x, scale.y, scale.z],
        'translation': [552.193724,1736.099481,235.409219]
    }
    core.update_model(id=neuron_mesh_model['id'], transformation=tf)

if load_astrocytes:
    astrocytes_sql_filter = 'sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200 AND guid%%6=0' % (scene_center[0], scene_center[1], scene_center[2])
    astrocytes_assembly_name = 'Astrocytes'
    be.remove_assembly(astrocytes_assembly_name)
    astrocytes_assembly = be.add_assembly(astrocytes_assembly_name)
    astrocytes_model = be.add_astrocytes(
        assembly_name=astrocytes_assembly_name,
        population_name=astrocytes_population_name,
        vasculature_population_name=vasculature_population_name,
        radius_multiplier=0.5,
        use_sdf=use_sdf, generate_internals=True,
        load_somas=True, load_dendrites=True,
        sql_filter=astrocytes_sql_filter, scale=scale)
    tf = {
        'rotation': [0, 0, 0, 1],
        'rotation_center': [0.0, 0.0, 0.0],
        'scale': [1,1,1],
        'translation': [ 20, 50, -80]
    }
    model_ids = be.get_model_ids()['ids']
    core.update_model(id=model_ids[len(model_ids)-1], transformation=tf)

if load_vasculature:
    vasculature_sql_filter = 'sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 300' % (scene_center[0], scene_center[1], scene_center[2])
    vasculature_assembly_name = 'Vasculature'
    be.remove_assembly(vasculature_assembly_name)
    vasculature_assembly = be.add_assembly(vasculature_assembly_name)
    vasculature_model = be.add_vasculature(
        assembly_name=vasculature_assembly_name,
        population_name=vasculature_population_name,
        representation=be.VASCULATURE_REPRESENTATION_SECTION,
        use_sdf=use_sdf,
        sql_filter=vasculature_sql_filter, scale=scale)
    tf = {
        'rotation': [0, 0, 0, 1],
        'rotation_center': [0.0, 0.0, 0.0],
        'scale': [1,1,1],
        'translation': [ 20, 50, -80]
    }
    model_ids = be.get_model_ids()['ids']
    core.update_model(id=model_ids[len(model_ids)-1], transformation=tf)


In [ ]:
def set_neuron_materials(model_id, opacity, color, shading_mode):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    reflection_indices = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        if mid == be.NEURON_MATERIAL_MITOCHONDRION:
            opacities.append(1.0)
            colors.append([0.3, 0.3, 0.3])
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(0.8)
            user_params.append(1.0 / scale.x)
            specular_exponents.append(60.0)
            emissions.append(0.5)
            reflection_indices.append(0.2)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            opacities.append(1.0)
            colors.append([0.9, 0.8, 0.8])
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.8)
            user_params.append(0.1 / scale.x)
            specular_exponents.append(4.0)
            emissions.append(0.2)
            reflection_indices.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA or mid==be.NEURON_MATERIAL_AXON:
            opacities.append(opacity)
            colors.append(color)
            shading_modes.append(shading_mode)
            glossinesses.append(0.75)
            user_params.append(0.05 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            opacities.append(1.0)
            colors.append([0.59, 0.82, 0.14])
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.1 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        else:
            # Membrane
            opacities.append(opacity)
            colors.append(color)
            shading_modes.append(be.SHADING_MODE_DIFFUSE)
            glossinesses.append(0.7)
            user_params.append(0.02 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
            
        refraction_indices.append(1.2)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        reflection_indices=reflection_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

def set_neuron_mesh_materials(model_id, opacity, color, shading_mode):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    reflection_indices = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        opacities.append(opacity)
        colors.append(color)
        shading_modes.append(shading_mode)
        glossinesses.append(0.5)
        user_params.append(0.2 / scale.x)
        specular_exponents.append(5.0)
        emissions.append(0.0)
        reflection_indices.append(0.0)
        refraction_indices.append(1.01)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        reflection_indices=reflection_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

def set_astrocyte_materials(model_id, opacity, color, shading_mode):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    reflection_indices = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        opacities.append(opacity)
        colors.append(color)
        shading_modes.append(shading_mode)
        glossinesses.append(0.1)
        user_params.append(0.01 / scale.x)
        specular_exponents.append(5.0)
        emissions.append(0.0)
        reflection_indices.append(0.0)
        refraction_indices.append(1.01)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        reflection_indices=reflection_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

def set_vasculature_materials(model_id):
    colors = list()
    opacities = list()
    refraction_indices = list()
    reflection_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for _ in material_ids:
        opacities.append(1.0)
        colors.append([1, 0, 0])
        shading_modes.append(be.SHADING_MODE_PERLIN)
        glossinesses.append(0.1)
        user_params.append(0.01 / scale.x)
        specular_exponents.append(3.0)
        refraction_indices.append(2.5)
        reflection_indices.append(0.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        reflection_indices=reflection_indices
    )

model_ids = be.get_model_ids()['ids']
index = 0
if load_neuron:
    set_neuron_materials(
        model_ids[index], 0.6, [206 / 256, 185 / 256, 135 / 256],
        be.SHADING_MODE_NONE)
    index += 1
if load_mesh_neuron:
    be.set_material_extra_attributes(model_ids[index])
    set_neuron_mesh_materials(
        model_ids[index], 0.6, [206 / 256, 185 / 256, 135 / 256],
        be.SHADING_MODE_PERLIN)
    index += 1
if load_astrocytes:
    set_astrocyte_materials(
        model_ids[index], 1.0, [206 / 256, 185 / 256, 135 / 256],
        be.SHADING_MODE_NONE)
    index += 1
if load_vasculature:
    set_vasculature_materials(model_ids[index])
status = core.set_renderer()

In [ ]:
status = be.reset_camera()
status = core.set_renderer()

From nucleus to bouton

In [ ]:
resource_folder = os.path.abspath('../../tests/test_files')

pdb_folder = os.path.join(resource_folder, 'pdb')
membrane_folder = os.path.join(pdb_folder, 'membrane')

lipids_folder = os.path.join(membrane_folder, 'lipids')
transporters_folder = os.path.join(pdb_folder, 'transporters')
tubules_folder = os.path.join(pdb_folder, 'tubules')

points = be.get_neuron_section_points(
    assembly_name=neuron_assembly_name, neuron_guid=neuron_guid, section_guid=0)

### Microtubules

In [ ]:
import numpy as np
import math

nb_microtubules = 3
microtubule_id = 0

tracks = [[111, -29, -73], [-10, -98, -3], [70, 50, 20]]

if load_microtubules:
    for j in range(nb_microtubules):
        for i in range(len(points) - 1):

            p0 = points[i]
            p1 = points[i+1]

            src = np.array(p0[:3])
            dst = np.array(p1[:3])
            forwardVector = np.subtract(dst, src)
            forward_length = np.sqrt(np.sum(forwardVector**2))
            microtubule_size  = Vector3(5.0, forward_length, 0.0)

            q = be.look_at(
                Vector3(p0[0], p0[1], p0[2]),
                Vector3(p1[0], p1[1], p1[2])
            )

            position = Vector3(
                tracks[j][0] + p0[0],
                tracks[j][1] + p0[1],
                tracks[j][2] + p0[2])
            
            name = 'Microtubule%05d' % microtubule_id
            microtubule_id += 1

            tubule = Protein(
                name=name + '_TUB',
                source=tubules_folder + '1tub.pdb', 
                rotation=Quaternion(0.0, 0.0, 0.707, 0.707),
                occurrences=microtubule_size.y * 7.0)

            volume = Volume(
                name=name,
                shape=be.ASSEMBLY_SHAPE_HELIX, shape_params=microtubule_size,
                protein=tubule)

            status = be.add_volume(
                position=position,
                rotation=q,
                volume=volume)

            status = be.set_protein_color_scheme(
                assembly_name=name,
                name=name + '_TUB',
                color_scheme=be.COLOR_SCHEME_CHAINS,
                palette_name='Set2', palette_size=5)

### Acetylcholin

In [ ]:
from bioexplorer import MolecularSystemAnimationParams

acetylcholin_path = os.path.join(pdb_folder, 'neuromodulation', 'acetylcholin.pdb')

# Molecules
nb_molecules = 5000
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS

def add_molecules(radius, frame):
    acetylcholin_assembly_name = 'Acetylcholin'
    acetylcholin_name = 'Acetylcholin'
    acetylcholin_position = Vector3(54080 + radius, 180960, 24110)
    acetylcholin = Protein(
        name=acetylcholin_name, 
        source=acetylcholin_path, load_non_polymer_chemicals=True, 
        occurrences=nb_molecules,
        animation_params=MolecularSystemAnimationParams(3, frame*2, 0.5, frame*2 + 1, 1.0)
    )
    volume = Volume(
        name=acetylcholin_assembly_name,
        shape=be.ASSEMBLY_SHAPE_FILLED_SPHERE, shape_params=Vector3(radius, 0, 0),
        protein=acetylcholin)
    be.add_volume(
        volume=volume, 
        representation=protein_representation,
        position=acetylcholin_position,
        atom_radius_multiplier=protein_radius_multiplier)
    model_ids = be.get_model_ids()['ids']
    model_id = model_ids[len(model_ids)-1]
    set_astrocyte_materials(model_id, 1.0, [0.5,0.8,0.7], be.SHADING_MODE_DIFFUSE)

## Movie

### Inside Axon, follow microtubule

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
import math

nb_frames = 100
aperture = 0.01
focus = 2.938
key_frames = list()

key_frames.append(
    {
        'apertureRadius': 0.0,
        'direction': [-0.5161824932867844, 0.71089422191883, 0.47768717678692263],
        'focusDistance': 1000000.0,
        'origin': [10 * 61645.09332918549, 10 * 164939.92821114144, 10 * 17640.52720466037],
        'up': [0.7637641783394794, 0.6344538040102345, -0.11888082462013372]
    }
)

key_frames.append(
    {
        'apertureRadius': 0.0,
        'direction': [-0.7166883232084326, 0.6973190499228045, -0.01019754830550157],
        'focusDistance': 1000000.0,
        'origin': [10 * 56185.91520320514, 10 * 172743.7060708133, 10 * 23601.54634817272],
        'up': [0.6915583632197203, 0.7087280378295366, -0.13946898814846398]
    }
)

key_frames.append(
    {
        'apertureRadius': 0.0,
        'direction': [-0.08489556391375896, 0.9184638069706967, -0.38628613554288993],
        'focusDistance': 1000000.0,
        'origin': [10 * 55063.99282926692, 10 * 173129.80736023423, 10 * 24155.774633378987],
        'up': [0.7637641783394794, 0.6344538040102345, -0.11888082462013372]
    }    
)

for i in range(0, len(points)-20, 5):
    origin = [
        points[i][0] - 15,
        points[i][1] + 15,
        points[i][2] - 15
    ]
    
    target = [
        points[i+1][0],
        points[i+1][1],
        points[i+1][2]
    ]

    dir = [0.0, 0.0, 0.0]
    ldir = 0.0
    for k in range(3):
        l = target[k] - origin[k]
        dir[k] = l * scale.to_list()[k]
        ldir += l * l
    for k in range(3):
        dir[k] /= math.sqrt(ldir)
    
    mm.set_camera(origin=origin, direction=dir, up=[0, 1, 0])
    key_frames.append(mm.get_camera())

key_frames.append(
{
    'apertureRadius': 0.0,
    'direction': [-0.7788156442562865, 0.46643004505381463, 0.4193914702670549],
    'focusDistance': 1000000.0,
    'origin': [10 * 54276.71983440195, 10 * 180841.0051283754, 10 * 24040.887191176796],
    'up': [0.29883854644002006, 0.8637709263664873, -0.4057034753679049]}
)

key_frames.append(
{
    'apertureRadius': 0.0,
    'direction': [-0.5701341231776339, 0.6236003424401975, 0.5348548349757523],
    'focusDistance': 1000000.0,
    'origin': [10 * 54220.26233495012, 10 * 180839.3287436113, 10 * 24015.522296933832],
    'up': [0.4420071457778133, 0.7816083500807779, -0.44013415019213764]}    
)

mm.build_camera_path(key_frames, nb_frames, nb_frames / 4.0)

In [ ]:
def set_rendering_settings(inside):
    params = core.BioExplorerRendererParams()
    params.exposure = 1
    params.max_bounces = 10
    params.epsilon_factor = 2.0
    params.fog_start = 100.0 * scale.x
    params.fog_thickness = 40.0 * scale.x
    params.gi_distance = 100.0 * scale.x

    if inside:
        params.gi_distance = 1.0 * scale.x
        params.fog_start = 5.0 * scale.x
        params.fog_thickness = 1.0 * scale.x

    params.gi_samples = 1
    params.gi_weight = 0.2
    params.shadows = 0.75
    params.soft_shadows = 0.25
    params.use_hardware_randomizer = True
    return core.set_renderer_params(params)

In [ ]:
set_rendering_settings(False)

In [ ]:
be.set_models_visibility(True)

## Rendering

In [ ]:
core.set_renderer(current='bio_explorer', subsampling=4, max_accum_frames=64)
set_rendering_settings(False)

In [ ]:
from tqdm import tqdm

k = 1
resolution = [k * 960, k * 540]
output_folder = '/tmp'
nb_frames = mm.get_nb_frames()
nb_diffusion_frames = 100
start_diffusion_frame = nb_frames - nb_diffusion_frames

core.set_renderer(current='bio_explorer', subsampling=4, max_accum_frames=64)
if not generate_movie:
    core.set_renderer(current='basic', subsampling=4, max_accum_frames=1)

for frame in tqdm(range(nb_frames)):
    inside = frame > 198 and frame < 703
    set_rendering_settings(inside)

    mm.set_current_frame(frame)
    if frame >= start_diffusion_frame:
        add_molecules(2 + frame - start_diffusion_frame, frame - start_diffusion_frame)
        be.set_models_visibility(True)

    if generate_movie:
        mm.create_snapshot(
            renderer='bio_explorer',
            size=resolution, samples_per_pixel=8,
            base_name='%05d' % frame,
            path=output_folder)
    else:
        core.set_renderer()
        import time
        time.sleep(0.1)